In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
import json
with open('/Users/brianschreffler/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
!pip install yelpapi

In [4]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
# set our API call parameters and filename before the first call
location = 'Port Charlotte'
term = 'Seafood'

In [7]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = f"{FOLDER}{term}-{location.split(',')[0]}.json"
JSON_FILE

'Data/Seafood-Port Charlotte.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/Seafood-Port Charlotte.json already exists.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=location,
                                term=term,
                               offset=n_results+1)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

117

In [13]:
results['businesses']


[{'id': 'UH67M7QseHepI-k1c6tjJA',
  'alias': 'brother-s-fish-house-port-charlotte',
  'name': 'Brother’s Fish House',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/7m798jPqiDD4qyH-MAnk5w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/brother-s-fish-house-port-charlotte?adjust_creative=IX0BPgmwhVVbGGNYDVN81A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=IX0BPgmwhVVbGGNYDVN81A',
  'review_count': 107,
  'categories': [{'alias': 'seafood', 'title': 'Seafood'},
   {'alias': 'sandwiches', 'title': 'Sandwiches'},
   {'alias': 'salad', 'title': 'Salad'}],
  'rating': 4.0,
  'coordinates': {'latitude': 26.97288, 'longitude': -82.0875},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '3821 Tamiami Trl',
   'address2': 'Ste A',
   'address3': '',
   'city': 'Port Charlotte',
   'zip_code': '33952',
   'country': 'US',
   'state': 'FL',
   'display_address': ['3821 Tamiami Trl',
    'Ste A',
    'Port Charlo

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [15]:
# how long is businesses?
len(results['businesses'])


20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

6

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=location,
                                        term=term, 
                                        offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    ## What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)


  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
df = pd.read_json(JSON_FILE)
df.head()


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,UH67M7QseHepI-k1c6tjJA,brother-s-fish-house-port-charlotte,Brother’s Fish House,https://s3-media3.fl.yelpcdn.com/bphoto/7m798j...,False,https://www.yelp.com/biz/brother-s-fish-house-...,107,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 26.97288, 'longitude': -82.0875}","[delivery, pickup]",$$,"{'address1': '3821 Tamiami Trl', 'address2': '...",+19416271006,(941) 627-1006,3778.615952
1,1CugTx4_Z1rD1Ley18ksFw,e-and-l-clam-house-port-charlotte,E & L Clam House,,False,https://www.yelp.com/biz/e-and-l-clam-house-po...,4,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",5.0,"{'latitude': 26.96931, 'longitude': -82.0637}",[],$$,"{'address1': '23440 Janice Ave', 'address2': N...",+19413915025,(941) 391-5025,6016.266885
2,HierAwL3ScsGPBFCvbyBxQ,rjs-new-england-seafood-port-charlotte,RJ's New England Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/6Tejue...,False,https://www.yelp.com/biz/rjs-new-england-seafo...,90,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 27.00664, 'longitude': -82.16296}",[delivery],$$,"{'address1': '17753 Toledo Blade Blvd', 'addre...",+19412350005,(941) 235-0005,4652.361693
3,qVNSgp34jdp8Tslye5FksA,captain-crab-seafood-and-sushi-port-charlotte,Captain Crab Seafood and Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/12TTgf...,False,https://www.yelp.com/biz/captain-crab-seafood-...,104,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 26.99077, 'longitude': -82.11019}","[delivery, pickup]",$$,"{'address1': '2746 Tamiami Trail', 'address2':...",+19416256888,(941) 625-6888,874.778864
4,-pnkbPNoIrTDgrHcjHW7rg,the-pier-at-fishermens-village-punta-gorda,The Pier at Fishermen's Village,https://s3-media1.fl.yelpcdn.com/bphoto/MC_ZSg...,False,https://www.yelp.com/biz/the-pier-at-fishermen...,286,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 26.9284874, 'longitude': -82.0628...","[pickup, delivery]",$$,"{'address1': '1200 W Retta Esplanade', 'addres...",+19416558267,(941) 655-8267,9009.385712


In [18]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file


'Data/Seafood-Port Charlotte.csv.gz'